# Example 1

In [1]:
# Working with matrices
import numpy as np
# Linear programming
from scipy.optimize import linprog

Two players play for money, simultaneously choosing one of the numbers 1 or 2, 
and then calculating the sum $S$. If $S$ is even, then the first player wins $S$ dollars from the second,
if $S$ is odd, then the second wins $S$ dollars from the first. 

Game matrix
$$
	A=\begin{pmatrix} 2 & -3 \\ -3 & 4\end{pmatrix}
$$

In [2]:
# Pay-off matrix
A = np.array([ [2, -3], [-3, 4] ])
print(A)

[[ 2 -3]
 [-3  4]]


## Upper and lower value of the game

\begin{align*}
    \overline{\nu}&=\min_{j}\max_i a_{ij} & \underline{\nu}&=\max_i\min_j a_{ij}
\end{align*}

In [3]:
# Lower value of the game
max( A.min(axis=1) )

-3

In [4]:
# Upper value of the game
min( A.max(axis=0) )

2

They are not equal $\Longrightarrow$ no Nash equilibrium in pure strategies

## Nash Equilibrium in pure strategies

In [5]:
Nash_eq = np.full(A.shape, 'No ')

for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        if (A[i,j]==max(A[:,j])) & (A[i,j]==min(A[i,:])):
            Nash_eq[i,j]='Yes'

print(Nash_eq)

[['No ' 'No ']
 ['No ' 'No ']]


## Expected Payoff
Suppose players follow mixed strategies 
\begin{align*}
    P^\top&=\begin{pmatrix} 0.3 & 0.7\end{pmatrix} & 
    Q^\top&=\begin{pmatrix} 0.25 & 0.75\end{pmatrix}.
\end{align*}
Expected payoff for each player:

- Expected payoff of the first player $EU_A(P,Q)=P^\top AQ$
- Expected payoff of the second player $EU_B(P,Q)=-EU_A(P,Q)$

In [6]:
P = np.array([0.3, 0.7])
Q = np.array([0.25, 0.75])
# Expected payoff. @ - matrix multiplication in numpy
P.T@A@Q

1.0499999999999998

## Nash Equilibrium in Mixed Strategies
The `scipy.optimize.linprog` library [solves](https://docs.scipy.org/doc/scipy/tutorial/optimize.html#linear-programming-linprog) linear programming problems in the form
\begin{gather*}
    \min c^\top x \\ s.t.\left\{\begin{aligned} 
    A_{ub}x &\leq b_{ub} \\ A_{eq}x &= b_{eq} \\ l\leq x&\leq u
    \end{aligned}\right.
\end{gather*}
**Important**: by default, `linprog` uses non-negativity constraints for variables

Matrix $A$ has negative elements $\Longrightarrow$ for the optimization problem, all elements of matrix $A$ need to be shifted by the same amount to make them positive.
For example, by 4 (variable `shift`).

### First player
The first player solves the optimization problem
\begin{gather*}
    \begin{gathered} \min 1^\top x\\ s.t. \left\{\begin{aligned}
    (A+4)^\top x &\geq 1 \\ x&\geq0
    \end{aligned}\right.
    \end{gathered} \Longrightarrow 
    \begin{gathered} \min c^\top x\\ s.t. \left\{\begin{aligned}
    A_{ub} x &\leq b_{ub} \\ 0&\leq x
    \end{aligned}\right.
    \end{gathered}
\end{gather*}
where $c^\top=\begin{pmatrix} 1 & 1 \end{pmatrix}$, $b^\top_{ub}=\begin{pmatrix} -1 & -1 \end{pmatrix}$, $A_{ub}=-A^\top-4$

Let in the minimization problem: $\hat{x}$ - optimal solution, $\hat{f}$ - optimal value of the objective function. Then

- Value of the game (expected payoff of the first) $1/\hat{f}-4$
- equilibrium strategy of the first player $P^*=\hat{x}/\hat{f}$

In [ ]:
c = np.array([1,1])
b = np.array([-1, -1])

shift = 4

res = linprog(c, A_ub=-A.T-shift, b_ub=b)

# value of the game, expected payoff of the first
value1 = 1/res.fun-shift
# equilibrium strategy
P_star = res.x/res.fun

P_star, value1

(array([0.58333333, 0.41666667]), -0.08333333333333304)

### Second player
The second player solves the optimization problem
\begin{gather*}
    \begin{gathered} \max 1^\top y\\ s.t. \left\{\begin{aligned}
    (A+4) y &\leq 1 \\ y&\geq0
    \end{aligned}\right.
    \end{gathered} \Longrightarrow 
    \begin{gathered} \min c^\top y\\ s.t. \left\{\begin{aligned}
    A_{ub} y &\leq b_{ub} \\ 0&\leq y
    \end{aligned}\right.
    \end{gathered}
\end{gather*}
where $c^\top=\begin{pmatrix} -1 & -1 \end{pmatrix}$, $b^\top_{ub}=\begin{pmatrix} 1 & 1 \end{pmatrix}$, $A_{ub}=A+4$

Let in the minimization problem: $\hat{x}$ - optimal solution, $\hat{f}$ - optimal value of the objective function. Then

- expected payoff of the second $1/\hat{f}+4$
- equilibrium strategy of the second player $Q^*=-\hat{x}/\hat{f}$

In [ ]:
c = np.array([-1, -1])
b = np.array([1, 1])

shift = 4

res = linprog(c, A_ub=A+shift, b_ub=b)

# expected payoff of the second = -value1
value2 = 1/res.fun+shift
# equilibrium strategy
Q_star = -res.x/res.fun

Q_star, value2

(array([0.58333333, 0.41666667]), 0.08333333333333393)